In [1]:
from glob import glob
from openai import OpenAI
from dotenv import load_dotenv
import os
import base64
import json

In [2]:
load_dotenv()
api_key=os.getenv("OPENAI_API_KEY")
client=OpenAI(api_key=api_key)

def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

In [3]:
def image_quiz(image_path, n_trial=0, max_trial=3):
    if n_trial >= max_trial:
        raise Exception ("Failed to generate a quiz,")
    
    base64_image = encode_image(image_path)

    quiz_prompt="""
    제공한 이미지를 바탕으로, 다음과 같은 양식으로 퀴즈를 만들어주세요.
    정답은 1~4 중 하나만 해당하도록 출제하요.
    토익 리스닝 문제 스타일로 문제를 만들어주세요.
    아래는 예시입니다.
    ---예시---

    Q: 다음 이미지에 대한 설명 중 옳지 않은 것은 무엇입니까?
    1. 베이커리에서 사람들이 빵을 사는 모습이 담겨 있습니다.
    2. 맨 앞에 서 있는 사람은 빨간색 셔츠를 입었습니다.
    3. 기차를 타기 위해 줄을 서 있는 사람들이 있습니다.
    4. 점원은 노란색 티셔츠를 입었습니다.

    Listening: Which of the following descriptions of the image is incorrect?
    -1. It shows people buying bread at a bakery.
    -2. The person standing at the front is wearing a red shirt.
    -3. There are people lining up to take a train.
    -4. The clerk is wearing a yellow T-shirt.

    정답 : 4. 점원은 노란색 티셔츠가 아닌 파란색 티셔츠를 입었습니다.
    (주의: 정답은 1~4중 하나만 선택하도록 출제하세요.)
    ===
    """

    messages=[
        {
            "role": "user",
            "content":[
                {
                    "type": "text",
                    "text": quiz_prompt
                },
                {
                    "type": "image_url",
                    "image_url":{
                        "url":f"data:image/jpeg;base64,{base64_image}",
                    },
                },
            ],
        }
    ]
    
    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=messages
        )
    except Exception as e:
        print("failed\n"+e)
        return image_quiz(image_path, n_trial+1)
    
    content = response.choices[0].message.content

    if "Listening:" in content:
        return content, True
    else:
        return image_quiz(image_path, n_trial+1)


In [ ]:
txt=''
eng_dict=[]
no=1
for g in glob('C:/llm01/gpt/images/*.jpg'):
    q, is_suceed = image_quiz(g)

    if not is_suceed:
        continue

    divider = f'## 문제 {no}\n\n'
    print(divider)

    txt += divider
    filename=os.path.basename(g)
    txt += f'![image]({filename})\n\n'

    print(q)
    txt += q + '\n\n----------------\n\n'

    with open('C:/llm01/gpt/images/image_quiz_eng.md','w',encoding='utf-8') as f:
        f.write(txt)

    #"Listening:" , "정답:" 기준으로 나눈뒤 영어부분만 가져오기
    eng=q.split('Listening:')[1].split('정답:')[0].strip()

    #리스트에 딕셔너리 형태로 정보 추가
    eng_dict.append({
        'no':no,
        'eng':eng,
        'img':filename
    })
    with open('C:/llm01/gpt/images/image_quiz_eng.json','w',encoding='utf-8') as f:
        json.dump(eng_dict, f, ensure_ascii=False, indent=4)
    #enshure_ascii : 한글도 깨지지 않게 저장, indent : 보기 좋게 들여쓰기 포함해서 저장
    no+=1

## 문제 1


Q: 다음 이미지에 대한 설명 중 옳지 않은 것은 무엇입니까?
1. 많은 사람들이 노트북을 사용하고 있습니다.
2. 행사장은 'DIVE 2024 IN BUSAN'이라는 배너가 보입니다.
3. 사람들은 체육관에서 운동 중입니다.
4. 천장에 여러 개의 조명이 설치되어 있습니다.

Listening: Which of the following descriptions of the image is incorrect?
-1. Many people are using laptops.
-2. There's a banner that says 'DIVE 2024 IN BUSAN'.
-3. People are exercising in a gym.
-4. Several lights are installed on the ceiling.

정답: 3. 사람들은 체육관에서 운동 중이 아닙니다.
## 문제 2


Q: 다음 이미지에 대한 설명 중 옳지 않은 것은 무엇입니까?
1. 노란색 조형물이 이미지의 왼쪽에 있습니다.
2. 건물 위에 "Local Stitch"라는 글자가 보입니다.
3. 빨간 벽돌 건물이 이미지 중앙에 있습니다.
4. 파란색 간판이 건물 앞에 있습니다.

Listening: Which of the following descriptions of the image is incorrect?
- 1. A yellow sculpture is on the left side of the image.
- 2. The words "Local Stitch" are visible on the building.
- 3. A red brick building is in the center of the image.
- 4. A blue sign is in front of the building.

정답: 4. 파란색 간판이 아닌 간판이 보이지 않습니다.
## 문제 3


Q: 다음 이미지에 대한 설명 중 옳지 않은 것은 무엇입니까?
1. 카페에는 노란색 벽이 있습니다.
2. 사